In [7]:
function sfr2(exisp, etasp, nnode)
    if (nnode == 3)
        shape = zeros(3)
        deriv = zeros(2,3)
        s = exisp
        t = etasp
        p = 1.0-s-t
        
        shape[1] = p
        shape[2] = s
        shape[3] = t
        
        deriv[1,1] = -1.0
        deriv[1,2] = 1.0
        deriv[1,3] = 0.0
        
        deriv[2,1] = -1.0
        deriv[2,2] = 0.0
        deriv[2,3] = 1.0
        
    end
    
    if(nnode == 4)
        # 4 nodes elements
        shape = zeros(4)
        deriv = zeros(2,4)
        
        s = exisp
        t = etasp
        st = s * t
        
        shape[1] = (1.0 - t-s+st)*0.25
        shape[2] = (1.0 - t+s-st)*0.25
        shape[3] = (1.0 + t+s+st)*0.25
        shape[4] = (1.0 + t-s-st)*0.25
        
        deriv[1,1] = (-1.0+t)*0.25
        deriv[1,2] = (1.0-t)*0.25
        deriv[1,3] = (1.0+t)*0.25
        deriv[1,4] = (-1.0+t)*0.25
        
        deriv[2,1] = (-1.0+s)*0.25
        deriv[2,2] = (-1.0-s)*0.25
        deriv[2,3] = (1.0+s)*0.25
        deriv[2,4] = (1.0-s)*0.25
    end
    
    if (nnode == 8)
        # -- 8 nodes elements
        shape = zeros(8)
        deriv = zeros(2,8)
        s = exisp
        t = etasp
        s2 = 2.0*s
        t2 = 2.0*t
        ss = s*s
        tt = t*t
        st = s*t
        sst = s*s*t
        stt = s*t*t
        st2 = 2.0*s*t
        
        shape[1] = (-1.0+st+ss+tt-sst-stt)*0.25
        shape[2] = (1.0-t-ss+sst)*0.5
        shape[3] = (-1.0-st+ss+tt-sst+stt)*0.25
        shape[4] = (1.0+s-tt-stt)*0.5
        shape[5] = (-1.0+st+ss+tt+sst+stt)*0.25
        shape[6] = (1.0+t-ss-sst)*0.5
        shape[7] = (-1.0-st+ss+tt+sst-stt)*0.25
        shape[8] = (1.0-s-tt+stt)*0.5
        
        deriv[1,1] = (t+s2-st2-tt)*0.25
        deriv[1,2] = -s+st
        deriv[1,3] = (-t+s2-st2+tt)*0.25
        deriv[1,4] = (1.0-tt)*0.5
        deriv[1,5] = (t+s2+st2+tt)*0.25
        deriv[1,6] = -s-st
        deriv[1,7] = (-t+s2+st2-tt)*0.25
        deriv[1,8] = (-1.0+tt)*0.5
        
        deriv[2,1] = (s+t2-ss-st2)*0.25
        deriv[2,2] = (-1.0+ss)*0.5
        deriv[2,3] = (-s+t2-ss+st2)*0.25
        deriv[2,4] = -t-st
        deriv[2,5] = (s+t2+ss+st2)*0.25
        deriv[2,6] = (1.0-ss)*0.5
        deriv[2,7] = (-s+t2+ss-st2)*0.25
        deriv[2,8] = -t+st
    end
    return shape, deriv
end

sfr2 (generic function with 1 method)

This function evaluates the Cartesian shape function derivatives, determinant of the Jacobian for the numerical intergration of area/volume of elements. and the Cartesian coordinates of the integration points within
elements.

In [4]:
function jacob2(ielem,elcod, kgasp, shape, deriv, nnode, ndime)
    # Gauss point coordinates
    gpcod = zeros(ndime,kgasp)
    
    for idime = 1 : ndime
        gpcod[idime, kgasp] = 0.0
        for inode = 1:nnode
            gpcod[idime, kgasp] = gpcod[idime,kgasp] + elcod[idime,inode]*shape[inode]
        end
    end
    
    # jacobian
    xjacm = zeros(ndime,ndime)
    for idime = 1:ndime
        for jdime = 1:ndime
            xjacm[idime,jdime] = 0.0
            for inode = 1:nnode
                xjacm[idime,jdime] = xjacm[idime,jdime]+deriv[idime,inode]*elcod[jdime,inode]
            end
        end
    end
    
    djacb = xjacm[1,1]*xjacm[2,2]-xjacm[1,2]*xjacm[2,1]
    
    if (djacb <= 0.0)
        println("Program terminated")
        println("Zero or negative area for element: ",ielem)
    end
    
    # Cartesian derivatives
    xjaci = zeros(ndime,ndime)
    xjaci[1,1] = xjacm[2,2]/djacb
    xjaci[2,2] = xjacm[1,1]/djacb
    xjaci[1,2] = -xjacm[1,2]/djacb
    xjaci[2,1] = -xjacm[2,1]/djacb
    
    cartd = zeros(ndime, nnode)
    for idime = 1 : ndime
        for inode = 1 : nnode
            cartd[idime,inode] = 0.0
            for jdime = 1 : ndime
                cartd[idime,inode] = cartd[idime,inode] + xjaci[idime,jdime]*deriv[jdime,inode]
            end
        end
    end
    return cartd, djacb, gpcod
end

jacob2 (generic function with 1 method)

In [6]:
function modps(mtype, ntype, nstre, props)
    # Material parameters
    young = props[mtype,1]
    poiss = props[mtype,2]
    
    dmatx = zeros(3,3)
    for istre = 1:3
        for jstre = 1:3
            dmatx[istre,jstre] = 0.0
        end
    end
    if (ntype == 1)
        # Plane stress
        consty = young/(1.0-poiss*poiss)
        dmatx[1,1] = consty
        dmatx[2,2] = consty
        dmatx[1,2] = consty*poiss
        dmatx[2,1] = consty*poiss
        dmatx[3,3] = (1.0-2.0*poiss)*consty/2.0
    end
    
    if (ntype == 2)
        # Plane Strain
        consty = young*(1.0 - poiss) / ((1+poiss)*(1.0-2.0*poiss))
        dmatx[1,1] = consty
        dmatx[2,2] = consty
        dmatx[1,2] = consty*poiss/(1.0 - poiss)
        dmatx[2,1] = consty*poiss/(1.0 - poiss)
        dmatx[3,3] = (1.0 - 2.0*poiss)*consty/(2.0*(1.0-poiss))
    end
    return dmatx
end

modps (generic function with 1 method)

This function forms the strain matrix by using the Cartesian derivatives of
the shape functions

In [2]:
function bmats(nevab, nstre, cartd, shape, nnode)
    ngash = 0
    bmatx = zeros(nstre, nevab)
    for inode = 1:nnode
        mgash = ngash + 1
        ngash = mgash + 1
        
        bmatx[1,mgash] = cartd[1,inode]
        bmatx[1,ngash] = 0.0
        bmatx[2,mgash] = 0.0
        bmatx[2,ngash] = cartd[2,inode]
        bmatx[3,mgash] = cartd[2,inode]
        bmatx[3,ngash] = cartd[1,inode]
    end
    return bmatx
end

bmats (generic function with 1 method)

This function multiplies the elasticity matrix and the strain matrix.

Variables and array list:
---------------------------------
    nevab: Total number of element variables
    nstre: Number of element variables (nevab = nnode x ndofn).
    bmatx: Strain matrix
    dmatx: Elasticity matrix.
    dbmat: Multiplication results of bmatx and dmatx

In [1]:
function dbe(nevab, nstre, bmatx, dmatx)
    dbmat = zeros(nstre,nevab)
    for istre = 1 : nstre
        for ievab = 1 : nevab
            dbmat[istre, ievab] = 0.0
            for jstre = 1 : nstre
                dbmat[istre, ievab] = dbmat[istre, ievab] + dmatx[istre,jstre]*bmatx[jstre,ievab]
            end
        end
    end
    return dbmat
end

dbe (generic function with 1 method)

This function forms the stiffness matrices at the element level first, then
assembles them into the global stiffness matrix.

In [ ]:
function stiffness(npoint, nelem, nnode, nstre, ndime, ndofn, 
    ngaus, ntype, lnods, matno, coord, props, posgp, weigp)
    
    # initialize global stiffness
    nevab = nnode*ndofn
    ngaus2 = ngaus
    if (nnode == 3)
        ngaus2 = 1
    end
    
    ntotv = npoin*ndofn
    gstif = zeros(ntotv, ntotv)
    estif = zeros(nevab, nevab)
    
    # Element stiffness and loads
    for ielem = 1 : nelem
        # Initialize element stiffness
        for ievab = 1 : nevab
            for jevab = 1 : nevab
                estif[ievab, jevab] = 0.0
            end
        end
        
        # Form elasticity matrix
        mtype = matno[ielem]
        dmatx = modps(mtype, ntype, nstre, props)
        
        # Coordinates of element nodes
        elcod = zeros(ndime,nnode)
        for inode = 1:nnode
            lnode = lnods[ielem, inode]
            for idime = 1 : ndime
                elcod[idime,inode] = coord[lnode, idime]
            end
        end
        
        # Integrate element stiffness
        kgasp = 0
        for igaus = 1 : ngaus
            exisp = posgp[igaus]
            for jgaus = 1 : ngaus2
                etasp = posgp[jgaus]
                if (nnode == 3)
                    etasp = posgp[ngaus + igaus]
                end
                
                kgasp = kgasp + 1
                shape, deriv = sfr2(exisp, etasp, nnode)
                cartd,djacb, gpcod = jacob2(ielem, elcod, kgasp,shape,deriv,nnode,ndime)
                bmatx = bmats(nevab, nstre,cartd,shape,nnode)
                dbmat = dbe(nevab, nstre, bmatx, dmatx)
                
                dvolu = djacb * weigp[igaus] * weigp[jgaus]
                
                if (nnode == 3)
                    dvolu = djacb * weigp[igaus]
                end
                
                # Form element stiffness
                for ievab = 1 : nevab
                    for jevab = 1 : nevab
                        for istre = 1 : nstre
                            estif[ievab,jevab] = estif[ievab,jevab] + bmatx[istre,ievab]*dbmat[istre, jevab]*dvolu
                        end
                    end
                end
            end
        end
        
        # Form global stiffness matrix
        for inode = 1:nnode
            lnode = lnods[ielem,inode]
            for idofn = 1 : ndofn
                itotv = (lnode - 1)*ndofn + idofn
                ievab = (inode - 1)*ndofn + idofn
                for jnode = 1 : nnode
                   knode = lnods[ielem, jnode]
                   for jdofn = 1 : ndofn
                       jtotv = (knode - 1)*ndofn + jdofn
                       jevab = (jnode - 1)*ndofn + jdofn
                        
                        gstif[itotv,jtotv] = gstif[itotv,jtotv] + estif[ievab, jevab]
                    end
                end
            end
        end
        
    end
    return gstif
end
    